In [ ]:
# 🏥 MEDICAL LLM LIVE DEMO
# This notebook loads the fine-tuned ChatDoctor model directly from Hugging Face.

# 1. Install dependencies (Quiet mode)
print("⏳ Installing libraries... (approx. 2 mins)")
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" -q
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes gradio -q

# 2. Load Model
from unsloth import FastLanguageModel
import torch
from transformers import TextStreamer
import gradio as gr

print("⬇️ Loading Model from Hugging Face...")

# YOUR ADAPTER MODEL ON HUGGING FACE
adapter_path = "khairi123/medical-phi35-chatdoctor"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = adapter_path, # Loads the base model AND your adapter automatically
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)
print("✅ Model Loaded Successfully!")

# 3. Define the Chat Function
def medical_chat(message, history):
    prompt = f"""Below is an instruction that describes a medical task. Write a response that appropriately completes the request.

### Instruction:
{message}

### Response:
"""
    inputs = tokenizer([prompt], return_tensors = "pt").to("cuda")

    streamer = TextStreamer(tokenizer, skip_prompt = True, skip_special_tokens = True)

    # Generate response
    outputs = model.generate(
        **inputs,
        streamer = streamer,
        max_new_tokens = 512,
        use_cache = True,
        temperature = 0.1,
        do_sample = True,
    )

    # We return an empty string here because the Streamer prints to console,
    # but for Gradio we need to decode.
    # Re-decoding for Gradio interface:
    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    return decoded.split("### Response:")[-1].strip()

# 4. Launch Gradio Interface
demo = gr.ChatInterface(
    fn=medical_chat,
    title="🏥 ChatDoctor - Medical AI Assistant",
    description="**Fine-tuned Phi-3.5 on 100k Medical Conversations.**\n\n⚠️ **Disclaimer:** This AI is for educational purposes only. Always consult a real doctor.",
    examples=["What are the symptoms of pneumonia?", "How to treat a fever at home?", "I have a sharp pain in my chest."],
    theme=gr.themes.Soft()
)

print("🚀 Launching Chatbot...")
demo.launch(share=True)